In [1]:
import pandas as pd
from main import *

**Source of test data**

Half-hour price data is taken from the GB day-ahead auction for 2022 sourced through N2EX. 

In [2]:
# test price curve
df = pd.read_excel('Data/auction-prices22.xls')
df = df.iloc[:,1:26]
df.rename(columns = df.iloc[4,:], inplace = True)
df.drop([0,1,2,3,4], inplace = True)
df.drop(['3B'], axis = 1, inplace = True)
df.ffill(inplace=True)
df = df.stack().reset_index().drop(columns = ['level_0','level_1']).set_axis(["Price (GBP/MWh)"], axis = 1)

hour = list(range(1,len(df)+1)) 
df.insert(loc=0, column = "Hour", value = hour)

time_range = pd.date_range(dt.datetime(2022, 1, 1), dt.datetime(2023,1,2), freq='1h')[1:]
df.set_index(time_range, inplace = True)

C:\Users\ovais\AppData\Local\Temp\ipykernel_23016\1923840718.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.ffill(inplace=True)


In [3]:
df.head()

,Hour,Price (GBP/MWh)
2022-01-01 01:00:00,1,115.95
2022-01-01 02:00:00,2,149.98
2022-01-01 03:00:00,3,99.99
2022-01-01 04:00:00,4,25.05
2022-01-01 05:00:00,5,7.85


**Parameters**

* t: timestep
* R_max: maximum power output/input
* S_max: maximum battery capacity
* D_max: maximum daily discharge volume (to represent daily cycles)
* F_charge: charge efficiency
* F_discharge: discharge efficiency
* Pt: spot price at time t
* S_t: storage at time t

**Decision variables**
* E_charge: Energy outtake at time t
* E_discharge: Energy intake at time t

**Constraints**
* St = St-1 + Eff_in*Ein - Eout/Eff_out
* for all t, St > 0
* for all t, St <= Smax
* for all t, Ein <= Rmax
* for all t, Eout <= Rmax
* for all t, Eout <= St




In [4]:
optimal_model = optimise_battery(df)

In [5]:
result_df = results(optimal_model)
result_df

,Hour,Price (GBP/MWh),Charge (MWh),Discharge (MWh),State of Charge (MWh)
0,1,115.95,1.0,0.0,0.000000
1,2,149.98,0.0,1.0,0.948683
2,3,99.99,0.0,0.0,0.000000
3,4,25.05,0.0,0.0,0.000000
4,5,7.85,0.0,0.0,0.000000
...,...,...,...,...,...
8779,8780,184.95,0.0,1.0,0.948683
8780,8781,164.00,0.0,0.0,0.000000
8781,8782,150.00,0.0,0.0,0.000000
8782,8783,150.01,0.0,0.0,0.000000
